# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/anthonyparry/Downloads/Starter_Code 9/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230



Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.


In [12]:
%%capture --no-display

import folium

# Create a map centered on a location of your choice (e.g., New York City)
map_city = folium.Map(location=[40.7128, -74.0060], zoom_start=5)

# Add a marker for each city in the DataFrame
for index, row in city_data_df.iterrows():
    humidity = row['Humidity']
    lat = row['Lat']
    lon = row['Lng']
    city_name = row['City']

    # Create a circle marker where the size represents the humidity
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the size as per your preference
        popup=f"{city_name}, Humidity: {humidity}%",
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(map_city)

# Display the map
map_city


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 20) \
    & (city_data_df["Wind Speed"] < 6.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
narrow_df = narrow_df.dropna()
# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
146,146,puerto del rosario,28.5004,-13.8627,24.86,73,0,4.63,ES,1666108035
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
81,makakilo city,US,21.3469,-158.0858,81,
146,puerto del rosario,ES,28.5004,-13.8627,73,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 20000
params = { 
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
puerto del rosario - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
pemba - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
capoterra - nearest hotel: No hotel found
stolac - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
san joaquin - nearest hotel: No hotel found
tikrit - nearest hotel: No hotel found
guerrero negro -

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
45,kapaa,US,22.0752,-159.3190,84,No hotel found
51,hilo,US,19.7297,-155.0900,83,No hotel found
63,banda,IN,25.4833,80.3333,52,No hotel found
73,nalut,LY,30.3333,10.8500,27,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,No hotel found
146,puerto del rosario,ES,28.5004,-13.8627,73,No hotel found
152,kahului,US,20.8947,-156.4700,60,No hotel found
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

import folium
import hvplot.pandas

# Create a function to generate a popup for each city marker
def create_popup(row):
    return f"{row['City']}, Humidity: {row['Humidity']}%"

# Create a map centered on a location of your choice (e.g., New York City)
map_city = folium.Map(location=[40.7128, -74.0060], zoom_start=5)

# Add a marker for each city in the DataFrame
for index, row in city_data_df.iterrows():
    humidity = row['Humidity']
    lat = row['Lat']
    lon = row['Lng']
    city_name = row['City']

    # Create a circle marker where the size represents the humidity
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity / 10,  # Adjust the size as per your preference
        popup=create_popup(row),
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(map_city)

# Display the map
map_city
